In [9]:
import numpy as np 
import matplotlib.pyplot as plt

import torch 
from torch import nn, optim 
import napari

import os 

import tifffile 

In [10]:
data_dir = '/home/confetti/mnt/data/processed/t1779/128roi_skip_gapped'
files = os.listdir(data_dir)
files = [file for file in files if file.endswith('.tif')]
files.sort()
files = [os.path.join(data_dir, file) for file in files]

print(len(files))
print(files[0])

128
/home/confetti/mnt/data/processed/t1779/128roi_skip_gapped/0001.tif


In [11]:
#for visualization
img = tifffile.imread(files[0])
thickness = 8
start =64
viewer = napari.Viewer(ndisplay=3)
for thick in range(2,16,2):
    projection = np.max(img[start:start + thick,:,:],axis=0)
    projection = np.squeeze(projection)
    viewer.add_image(projection, name = f"{thick}_mip")
viewer.add_image(img)
viewer.add_image(img[start], name = f"0_mip")

<Image layer '0_mip' at 0x7fd679526ff0>

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = os.listdir(data_dir)
        self.files = [file for file in self.files if file.endswith('.tif')]
        self.files.sort()
        self.files = [os.path.join(data_dir, file) for file in self.files]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = tifffile.imread(self.files[idx])
        img = np.array(img).astype(np.float32)
        img = torch.from_numpy(img)
        return img.unsqueeze(0)


In [13]:
class AutoEncoder3D(nn.Module):
    def __init__(self, in_channels=1, base_channels=32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, base_channels, kernel_size=3, stride=2, padding=1),  # downsample by 2 
            nn.LeakyReLU(),
            nn.Conv3d(base_channels, base_channels*2, kernel_size=3, stride=2, padding=1),  # downsample by 2
            nn.LeakyReLU(),
            nn.Conv3d(base_channels*2, base_channels*2, kernel_size=3, stride=2, padding=1),    # downsample by 2
            nn.LeakyReLU(),
            nn.Conv3d(base_channels*2, base_channels, kernel_size=3, stride=2, padding=1),    # downsample by 2
            nn.LeakyReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(base_channels, base_channels*2, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose3d(base_channels*2, base_channels*2, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose3d(base_channels*2, base_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose3d(base_channels, in_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU()
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoEncoder3D(1, 32).to(device)
dataset = Dataset(data_dir)

from torch.utils.data import DataLoader
loader  = DataLoader(dataset,batch_size=8,shuffle=True) 
inputs = dataset[0].to(device)

output = model(inputs)
inputs.shape, output.shape

(torch.Size([1, 128, 128, 128]), torch.Size([1, 128, 128, 128]))

In [21]:
from tqdm import tqdm
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
epochs = 500
for epoch in range(epochs):
    for data in tqdm(loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs} Loss: {loss.item()}')

100%|██████████| 16/16 [00:11<00:00,  1.40it/s]


Epoch 1/500 Loss: 26264.9140625


100%|██████████| 16/16 [00:07<00:00,  2.02it/s]


Epoch 2/500 Loss: 19427.265625


100%|██████████| 16/16 [00:08<00:00,  2.00it/s]


...

Epoch 110/500 Loss: 5290.2646484375


100%|██████████| 16/16 [00:07<00:00,  2.02it/s]


Epoch 111/500 Loss: 6091.1015625


  6%|▋         | 1/16 [00:00<00:11,  1.35it/s]


KeyboardInterrupt: 

In [22]:
model.eval()

batch = next(iter(loader))
batch = batch.to(device)
preds = model(batch)
preds = preds.detach().cpu()
preds = np.squeeze(preds)
batch = batch.detach().cpu()
batch = np.squeeze(batch)

idx  = 0
pred = preds[idx]
input = batch[idx]
viewer.add_image(pred,name=f"recon_{idx}")
viewer.add_image(input,name=f"input_{idx}")

pred = np.max(pred,axis=0)
input = np.max(input,axis=0)

fig, axs =plt.subplots(3)
img1 = axs[0].imshow(input,cmap = 'viridis')
axs[0].set_title("input")
fig.colorbar(img1,ax = axs[0])
img2 = axs[1].imshow(pred,cmap = 'viridis')
axs[1].set_title("pred")
fig.colorbar(img2,ax = axs[1])
img3 = axs[2].imshow(input - pred,cmap = 'viridis')
axs[2].set_title("residual")
fig.colorbar(img3,ax = axs[2])
    


TypeError: max() received an invalid combination of arguments - got (out=NoneType, axis=int, ), but expected one of:
 * ()
 * (Tensor other)
 * (int dim, bool keepdim = False)
      didn't match because some of the keywords were incorrect: out, axis
 * (name dim, bool keepdim = False)
      didn't match because some of the keywords were incorrect: out, axis
